In [6]:
from lxml import etree
import pickle
import pandas as pd
import nltk.data
import re
from nltk.tokenize import word_tokenize

In [140]:
from itertools import islice
def n_grams(a, n):
    z = (islice(a, i, None) for i in range(n))
    return zip(*z)

In [2]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [193]:
xml = open('data/1.1.text.xml', encoding='utf8').read()

In [194]:
tree = etree.fromstring(xml)

In [195]:
texts = tree.getchildren()

In [196]:
articles = []
for text in texts:
    title, abstract = text.getchildren()
    
    title = title.text
    ab_text = abstract.itertext()
    abstract = list(abstract)
    sequence = []
    for ab in ab_text:
        for el in abstract:
            if el.text == ab:
                sequence.append((ab, el.get('id')))
                _ = abstract.pop(abstract.index(el))
                break
        else:
            sequence.append((ab, 'O'))
    articles.append((title, sequence))

In [110]:
pickle.dump(articles, open('train_articles.pkl', 'wb'))

In [47]:
articles[0]

('Activity detection for information access to oral communication',
 [(' ', 'O'),
  ('Oral communication', 'H01-1001.1'),
  (' is ubiquitous and carries important information yet it is also time consuming to document. Given the development of ',
   'O'),
  ('storage media and networks', 'H01-1001.2'),
  (' one could just record and store a ', 'O'),
  ('conversation', 'H01-1001.3'),
  (' for documentation. The question is, however, how an interesting information piece would be found in a ',
   'O'),
  ('large database', 'H01-1001.4'),
  (' . Traditional ', 'O'),
  ('information retrieval techniques', 'H01-1001.5'),
  (' use a ', 'O'),
  ('histogram', 'H01-1001.6'),
  (' of ', 'O'),
  ('keywords', 'H01-1001.7'),
  (' as the ', 'O'),
  ('document representation', 'H01-1001.8'),
  (' but ', 'O'),
  ('oral communication', 'H01-1001.9'),
  (' may offer additional ', 'O'),
  ('indices', 'H01-1001.10'),
  (' such as the time and place of the rejoinder and the attendance. An alternative ',
   '

In [53]:
spans= sent_detector.span_tokenize(s)

In [73]:
sent_detector._params.abbrev_types.add('i.e')

In [79]:
relations = []
for article in articles:
    text = ''.join([text[0] for text in article[1]])
    i = 0
    entities = []
    for seq in article[1]:
        if seq[1] != 'O':
            entities.append((i, i+len(seq[0]), seq[1]))
        i += len(seq[0])
    sent_spans = sent_detector.span_tokenize(text)
    article_rels = []
    for span in sent_spans:
        sent_ents = []
        for entity in entities:
            span_range = set(range(span[0], span[1]))
            entity_range = set(range(entity[0], entity[1]))
            if (span_range&entity_range) and not span_range.issuperset(entity_range):
                print('entity overlaps sentences')
                print(text[span[0]:span[1]])
                print(text[entity[0]:entity[1]])
                print(entity)
                raise KeyError
            elif (span_range & entity_range) and span_range.issuperset(entity_range):
                sent_ents.append((text[entity[0]:entity[1]], entity[2]))
        article_rels.append((text[span[0]:span[1]], sent_ents))
    relations.append((article[0], article_rels))
            

In [104]:
# relations[2]

In [197]:
rels = {}
for article in relations:
    title, sents = article
    for sent in sents:
        if len(sent[1]) < 2:
            continue
        for i in range(len(sent[1])):
            for j in range(1, len(sent[1][i+1:])+1):
                start = sent[0].find(sent[1][i][0]) + len(sent[1][i][0])
                end = sent[0].find(sent[1][i+j][0])
                if start > end:
                    context = sent[0][end:start - len(sent[1][i][0])]
                else:
                    context = sent[0][start:end]
                rels[(sent[1][i][1], sent[1][i+j][1])] = [sent[0], context, sent[1][i][0], sent[1][i+j][0]]

In [12]:
# rels

In [48]:
s = ''.join([text[0] for text in articles[0][1]])


In [9]:
texts = []
for article in articles:
    text = []
    for t in article[1]:
        if t[1] == 'O':
            text.append(t[0])
        else:
            text.append(t[0].replace(' ', '_') + '_' + t[1])
    text = ''.join(text)
    sents = sent_detector.tokenize(text)
    texts.append((article[0], sents))

In [45]:
relations = []
bad = []
for article in texts:
    
    for sent in article[1]:
#         entities = re.findall('[\w\.\-\(\)\[\]\'\/\,\=\#]+(?:_[\w\.\-\(\)\[\]\'\/\,\=\#]+)+', sent)
        entities = re.findall('[\w\.\-\(\)\[\]\'\/\,\=\#\*"\+\&\%]+(?:_[\w\.\-\(\)\[\]\'\/\,\=\#\*"\+\&\%]+)*?(?:_+[A-Z][0-9][0-9]\-[0-9]{3,5}\.[0-9]{1,3}(?:[\w\,\;\:\(\)\"\*]*)?)', sent)
        rel = []
        words = sent.split()
        if len(entities) > 1:
            for i in range(len(entities)):
                for j in range(1, len(entities[i+1:])):
#                     start = sent.find(entities[i]) + len(entities[i]) + 1
#                     end = sent.find(entities[i+j])
                    start = words.index(entities[i]) + 1
                    end = words.index(entities[i+j])

                    context = words[start:end]
                    clear_context = []
                    for word in context:
                        if '_' in word:
                            w = word.split('_')[:-1]
                            clear_context += w
                        else:
                            clear_context += [word]
                    rel.append((sent, entities[i], context, entities[i+j]))
        
        relations += rel
#         else:
#             bad.append(sent)

ValueError: '_precision_P05-1076.20' is not in list

In [46]:
words

['The',
 'experiments_P05-1076.17',
 'show',
 'that',
 'the',
 'system_P05-1076.18',
 'is',
 'able',
 'to',
 'detect',
 'scf_types_P05-1076.19',
 'with',
 '70%_precision_P05-1076.20',
 'and',
 '66%_recall_rate_P05-1076.21.']

In [18]:
f = open('bad.txt', 'w', encoding='utf8')
for rel in bad:
#     f.write('#\t#'.join(rel)+'\n')
    f.write(rel+'\n')
f.close()

In [199]:
gold_relations = [re.findall('([A-Z]+)\(([\w\.\-]+)\,([\w\.\-]+)(?:\,([\w\.\-]+))?\)', text) 
                  for text in open('data/1.1.relations.txt', encoding='utf8').read().splitlines()]


In [200]:
rel_dict_golden = {}
for rel in gold_relations:
    rel = rel[0]
    rel_dict_golden[(rel[1], rel[2])] = [rel[0], rel[3]]


In [201]:
train_rels = {}
for rel in rels:
    if rel in rel_dict_golden:
        train_rels[rel] = rels[rel] + rel_dict_golden[rel]
    else:
        train_rels[rel] = rels[rel] + ['NO_RELATION', '']

In [120]:
len(train_rels)

8675

In [13]:
# train_rels

In [209]:
f = open('train_data_view.tsv', 'w', encoding='utf8')
f.write('tag\tentity1\tentity2\tcontext\tsent\treverse\n')
for rel in train_rels:
    s = re.sub('\n', '', '###'.join([train_rels[rel][4], train_rels[rel][2], train_rels[rel][3], 
                       train_rels[rel][1], train_rels[rel][0], train_rels[rel][5]]))
#     if len(s.split('\t')) != 6:
#         print(rel)
#         print(len(s.split('\t')))
#         raise KeyError
    f.write(s+'\n')
f.close()

In [158]:
rel = ('E06-1004.11', 'E06-1004.13')

In [165]:
train_rels[rel]

['Observing that the quality of the lexicon greatly impacts the accuracy that can be achieved by the algorithms, we present a method of HMM training that improves accuracy when training of lexical probabilities is unstable.',
 ' greatly impacts the accuracy that can be achieved by the algorithms, we present a method of HMM training that improves accuracy when training of ',
 'lexicon',
 'lexical probabilities',
 'NO_RELATION',
 '']

In [202]:
targets = []
entitys1 = []
entitys2 = []
contexts = []
sents = []
reverse = []
for rel in train_rels:
    targets.append(train_rels[rel][4])
    entitys1.append(train_rels[rel][2])
    entitys2.append(train_rels[rel][3])
    contexts.append(train_rels[rel][1])
    sents.append(train_rels[rel][0])
    reverse.append(train_rels[rel][5])

In [203]:
data = pd.DataFrame({'target':targets, 'ent1':entitys1, 'ent2':entitys2,
                     'context': contexts, 'sent':sents, 'reverse':reverse})

In [204]:
data.head()

,context,ent1,ent2,reverse,sent,target
0,and average retrieval times for looking up,computational complexity,phrase translations,,We detail the computational complexity and ave...,NO_RELATION
1,", we have examined a",compositional classes of paraphrases,class-oriented framework,,\nTowards deep analysis of compositional class...,NO_RELATION
2,"), and the state of focus of attention (calle...",intentional structure,attentional state,,"In this theory, discourse structure is compose...",NO_RELATION
3,", building",text,unambiguous structures,REVERSE,Sequences of cascades of rules deterministical...,FEATURE
4,designed for,features,pronoun resolution,,We present a set of features designed for pron...,NO_RELATION


In [205]:
data.to_csv('train_data.tsv', sep='\t', index=None, encoding='utf8')

In [2]:
d = pd.read_csv('train_data.tsv', '\t')

In [7]:
d

,context,ent1,ent2,reverse,sent,target
0,and average retrieval times for looking up,computational complexity,phrase translations,NaN,We detail the computational complexity and ave...,NO_RELATION
1,", we have examined a",compositional classes of paraphrases,class-oriented framework,NaN,\r\nTowards deep analysis of compositional cla...,NO_RELATION
2,"), and the state of focus of attention (calle...",intentional structure,attentional state,NaN,"In this theory, discourse structure is compose...",NO_RELATION
3,", building",text,unambiguous structures,REVERSE,Sequences of cascades of rules deterministical...,FEATURE
4,designed for,features,pronoun resolution,NaN,We present a set of features designed for pron...,NO_RELATION
5,words differs in,IR,words,NaN,"Since the significance of words differs in IR,...",NO_RELATION
6,of word n-grams (87.2%-96.7% accuracy dependi...,frequency counts,classification method,NaN,\r\nThis paper shows that it is very often pos...,NO_RELATION
7,"are presented, along with a control structure...",entity-oriented language definition,entity-oriented parser,NaN,Representative samples from an entity-oriented...,NO_RELATION
8,on the,accuracy,Penn Treebank WSJ,NaN,"Using these ideas together, the resulting tagg...",NO_RELATION
9,description that serves a specific purpose in ...,automatically generated text,description,NaN,We discuss the underlying relationship between...,NO_RELATION


In [208]:
d.target.value_counts()

NO_RELATION    7450
USAGE           482
FEATURE         326
WHOLE           232
COMPARE          95
RESULT           72
TOPIC            18
Name: target, dtype: int64

In [1]:
d['sent'][d['target'] == 'COMPARE']

NameError: name 'd' is not defined

### Неразмеченные тексты


In [3]:
import os
import flashtext
import re

In [4]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
punkt_param = PunktParameters()
abbreviation = ['al', 'fig', 'w.r.t', 'eq', 'e.g', 'i.e']
punkt_param.abbrev_types = set(abbreviation)
sent_tokenize = PunktSentenceTokenizer(punkt_param)


In [5]:
files = ['unlabeled/'+file for file in os.listdir('unlabeled')]


In [6]:
text = open(files[1], encoding='utf8').read()

In [7]:
sents = [re.sub('\n', '', sent) for sent in sent_tokenize.tokenize(text) if len(sent) > 40 and len(re.findall('\n', sent)) < 4 and not sent.startswith('[')]

In [244]:
# sent_tokenize.tokenize(text)

In [8]:
sents

['Less is More: Micro-expression Recognition from Video using Apex FrameSze-Teng Lionga, John Seeb, KokSheik Wongc,∗, Raphael C.-W.',
 'Conventional feature extraction approaches for micro-expression video considereither the whole video sequence or a part of it, for representation.',
 'However, with the high-speed video capture of micro-expressions (100-200 fps), are all frames necessary to provide a suﬃciently meaningful representation?',
 'Is the luxury ofdata a bane to accurate recognition?',
 'A novel proposition is presented in this paper, whereby we utilize only two imagesper video, namely, the apex frame and the onset frame.',
 'The apex frame of a video contains the highest intensity ofexpression changes among all frames, while the onset is the perfect choice of a reference frame with neutral expression.',
 'A new feature extractor, Bi-Weighted Oriented Optical Flow (Bi-WOOF) is proposed to encode essential expressivenessof the apex frame.',
 'We evaluated the proposed method o

In [62]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
# keyword_processor.add_keyword(<unclean name>, <standardised name>)
keyword_processor.add_keyword('used for')

#keywords_found = keyword_processor.extract_keywords('I love Big Apple and Bay Area.')

True

In [63]:
context_wl = []
for file in files:
    text = open(file, encoding='utf8').read()
    sents = [re.sub('\n', '', sent) for sent in sent_tokenize.tokenize(text) if len(sent) > 40 and len(re.findall('\n', sent)) < 4 and not sent.startswith('[')]
    for sent in sents:
        if keyword_processor.extract_keywords(sent):
            context_wl.append(sent)

In [64]:
context_wl

['We follow a 5-fold cross-validation scheme whereeleven subjects are selected for training and the remaining two subjects are used for testing.',
 'The left plot of Fig. 7 shows the phase transition of iPursuitin the plane of n1 and n2 and (IP) is used for subspaceidentiﬁcation.',
 'There are manyconstructions that can be used for this purpose, with different constructions leading to different adaptive algorithms [26]–[29].',
 'This asymmetry will be shown later, e.g., in Example VII.4, to be problematic when the consensusstrategy is used for adaptation and learning over networks.',
 'Nevertheless, we are not aware of similar coresets for poseestimation of kinematic data, or low-cost wireless trackingsystems that can be used for hovering of a very unstablequadcopter in dozens of frames per second.',
 'Notation: Lowercase boldface letters represent vectors (x)and uppercase boldface letters are used for matrices (X).',
 'All these M-step results and similar such results for many other m

In [3]:
import spacy
from nltk import Tree

In [2]:
# list(zip(np.random.uniform(0, 1, 100), np.random.uniform(10, 29, 100)))

In [4]:
nlp = spacy.load('en')

In [44]:
l = iter(range(100))
f = lambda: next(l)

In [59]:
d[d['target'] != 'NO_RELATION']['sent'].iloc[f()]

'So, for any place where the easily identifiable fragments occur in the sentence , the process will extend to both the left and the right of the islands , until possibly completely missing fragments are reached.'

In [4]:
sent = 'This correspondence can be used for providing a direct interpretationof Markov properties, both in the undirected and directed case, allowing an enhanced manipu-lation of these Markov models.'

In [6]:
p = nlp(sent)

In [9]:


def to_conll(sent):
    t = []
    for i, word in enumerate(sent):
#         if word.head is word:
#             head_idx = 0
#         else:
#              head_idx = word.i-sent[i].i+1
        head_idx = word.head.i+1
        t.append((
            str(i+1),
            word.lemma_,
            '_',# There's a word.i attr that's position in *doc*
            word.pos_,
            '_',# Coarse-grained tag
            word.tag_, # Fine-grained tag
            str(head_idx) if word.dep_ != 'ROOT' else str(0),
            word.dep_, # Relation
            '_', '_'))
    return '\n'.join(['\t'.join(s) for s in t])



In [12]:
f = open('trees_golden.txt', 'w')
for line in d[d['target'] != 'NO_RELATION']['sent']:
    l = re.sub('[\r\n]', '', line)
    l = l.lstrip(' ')
    doc = nlp(l)
    
    conll = to_conll(doc)
    f.write(conll+'\n\n')
f.close()

In [7]:
list(p.noun_chunks)

[This correspondence,
 a direct interpretationof Markov properties,
 the undirected and directed case,
 an enhanced manipu-lation,
 these Markov models]

In [16]:
d.ent1[0]

'computational complexity'

In [17]:
d.ent2[0]

'phrase translations'

In [7]:
import os
os.getcwd()

'/home/mnefedov/Рабочий стол/phd'

In [21]:
len(p.print_tree()[0])

7

In [23]:
list(p.sents)[0].root

applied

In [8]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_


t = to_nltk_tree(list(p.sents)[0].root)

In [11]:
t = []
for sent in p.sents:
    for i, word in enumerate(sent):
#         if word.head is word:
#             head_idx = 0
#         else:
#              head_idx = word.i-sent[i].i+1
        head_idx = word.head.i+1
        t.append((
            str(i+1),
            word.lemma_,
            '_',# There's a word.i attr that's position in *doc*
            word.pos_,
            '_',# Coarse-grained tag
            word.tag_, # Fine-grained tag
            str(head_idx) if word.dep_ != 'ROOT' else str(0),
            word.dep_, # Relation
            '_', '_'))

In [88]:
word.head.i

5

In [12]:
dp = '\n'.join(['\t'.join(s) for s in t])

In [13]:
t

[('1', 'this', '_', 'DET', '_', 'DT', '2', 'det', '_', '_'),
 ('2', 'correspondence', '_', 'NOUN', '_', 'NN', '5', 'nsubjpass', '_', '_'),
 ('3', 'can', '_', 'VERB', '_', 'MD', '5', 'aux', '_', '_'),
 ('4', 'be', '_', 'VERB', '_', 'VB', '5', 'auxpass', '_', '_'),
 ('5', 'use', '_', 'VERB', '_', 'VBN', '0', 'ROOT', '_', '_'),
 ('6', 'for', '_', 'ADP', '_', 'IN', '5', 'prep', '_', '_'),
 ('7', 'provide', '_', 'VERB', '_', 'VBG', '6', 'pcomp', '_', '_'),
 ('8', 'a', '_', 'DET', '_', 'DT', '12', 'det', '_', '_'),
 ('9', 'direct', '_', 'ADJ', '_', 'JJ', '12', 'amod', '_', '_'),
 ('10', 'interpretationof', '_', 'NOUN', '_', 'NN', '12', 'amod', '_', '_'),
 ('11', 'markov', '_', 'PROPN', '_', 'NNP', '12', 'compound', '_', '_'),
 ('12', 'property', '_', 'NOUN', '_', 'NNS', '7', 'dobj', '_', '_'),
 ('13', ',', '_', 'PUNCT', '_', ',', '7', 'punct', '_', '_'),
 ('14', 'both', '_', 'CCONJ', '_', 'CC', '15', 'advmod', '_', '_'),
 ('15', 'in', '_', 'ADP', '_', 'IN', '7', 'prep', '_', '_'),
 ('16', 't

In [68]:
deps_on_root = []
root = ''
for i, word, _, pos, _, _, head, rel, _, _ in t:
    if rel == 'ROOT':
        root = i

for i, word, _, pos, _, _, head, rel, _, _ in t:
    if head == root:
        if rel != 'punct':
            deps_on_root.append((i, str(word), rel))
            
def subtree(deps, h):
    if h not in deps:
        return h
    else:
        return [subtree(deps, sh) for sh in deps[h]] + [h]
    

def flatten(ll):
    if not isinstance(ll, list):
        return [ll]
    
    fl = []
    for el in ll:
        if not isinstance(el, list):
            fl += [el]
        else:
            fl += flatten(el)
    return sorted(fl, key=lambda x: int(x))

def get_seqs(subtrees):
    seqs = []
    for st in subtrees:
        seqs.append([id2words[j] for j in flatten(st)])
    return seqs
        

In [72]:
deps = defaultdict(list)
id2words = {}
id2rel = {}
for i, word, _, pos, _, _, head, rel, _, _ in t:
#     if rel == 'punct':
#         continue
    deps[head].append(i)
    id2words[i] = word

deps = dict(deps)

In [42]:
deps_on_root

[('2', 'correspondence', 'nsubjpass'),
 ('3', 'can', 'aux'),
 ('4', 'be', 'auxpass'),
 ('6', 'for', 'prep')]

In [74]:
get_seqs(subtree(deps, '6'))

[['provide',
  'a',
  'direct',
  'interpretationof',
  'markov',
  'property',
  ',',
  'both',
  'in',
  'the',
  'undirected',
  'and',
  'direct',
  'case',
  ',',
  'allow',
  'an',
  'enhanced',
  'manipu',
  '-',
  'lation',
  'of',
  'these',
  'markov',
  'model'],
 ['for']]

In [73]:
subtree(deps, '6')

[[['8', '9', '10', '11', '12'],
  '13',
  ['14', ['16', ['18', '19', '17'], '20'], '15'],
  '21',
  [['23', '24', '25', '26', [['29', '30', '31'], '28'], '27'], '22'],
  '7'],
 '6']

In [59]:
a = flatten(['14', ['16', ['18', '19', '17'], '20'], '15'])

In [60]:
[id2words[j] for j in a]

['both', 'in', 'the', 'undirected', 'and', 'direct', 'case']

In [1]:
dp = '1\t-PRON-\t_\tPRON\t_\tPRP\t1\tnsubj\t_\t_\n2\tdetail\t_\tVERB\t_\tVBP\t0\tROOT\t_\t_\n3\tthe\t_\tDET\t_\tDT\t3\tdet\t_\t_\n4\tcomputational\t_\tADJ\t_\tJJ\t4\tamod\t_\t_\n5\tcomplexity\t_\tNOUN\t_\tNN\t5\tdobj\t_\t_\n6\tand\t_\tCCONJ\t_\tCC\t6\tcc\t_\t_\n7\taverage\t_\tADJ\t_\tJJ\t7\tamod\t_\t_\n8\tretrieval\t_\tNOUN\t_\tNN\t8\tconj\t_\t_\n9\ttime\t_\tNOUN\t_\tNNS\t9\tnpadvmod\t_\t_\n10\tfor\t_\tADP\t_\tIN\t10\tprep\t_\t_\n11\tlook\t_\tVERB\t_\tVBG\t11\tpcomp\t_\t_\n12\tup\t_\tPART\t_\tRP\t12\tprt\t_\t_\n13\tphrase\t_\tNOUN\t_\tNN\t13\tcompound\t_\t_\n14\ttranslation\t_\tNOUN\t_\tNNS\t14\tdobj\t_\t_\n15\tin\t_\tADP\t_\tIN\t15\tprep\t_\t_\n16\t-PRON-\t_\tADJ\t_\tPRP$\t16\tposs\t_\t_\n17\tsuffix\t_\tADJ\t_\tJJ\t17\tamod\t_\t_\n18\tarray\t_\tNOUN\t_\tNN\t18\tnpadvmod\t_\t_\n19\t-\t_\tPUNCT\t_\tHYPH\t19\tpunct\t_\t_\n20\tbase\t_\tVERB\t_\tVBN\t20\tamod\t_\t_\n21\tdata\t_\tNOUN\t_\tNN\t21\tcompound\t_\t_\n22\tstructure\t_\tNOUN\t_\tNN\t22\tpobj\t_\t_\n23\t.\t_\tPUNCT\t_\t.\t23\tpunct\t_\t_'

In [49]:
from nltk.grammar import DependencyGrammar
from nltk.parse import (DependencyGraph, ProjectiveDependencyParser, NonprojectiveDependencyParser)
# import matplotlib.pyplot as plt
# %matplotlib inline
from collections import defaultdict
from itertools import chain

In [16]:
dg = DependencyGraph(dp)

In [68]:
p.

(HMM, 45)

In [24]:
dg.left_children(8)

0

In [27]:
dg.nodes

defaultdict(<function nltk.parse.dependencygraph.DependencyGraph.__init__.<locals>.<lambda>>,
            {0: {'address': 0,
              'ctag': 'TOP',
              'deps': defaultdict(list, {'ROOT': [5]}),
              'feats': None,
              'head': None,
              'lemma': None,
              'rel': None,
              'tag': 'TOP',
              'word': None},
             1: {'address': 1,
              'ctag': 'DET',
              'deps': defaultdict(list, {}),
              'feats': 'DT',
              'head': 2,
              'lemma': '_',
              'rel': 'det',
              'tag': '_',
              'word': 'this'},
             2: {'address': 2,
              'ctag': 'NOUN',
              'deps': defaultdict(list, {'det': [1]}),
              'feats': 'NN',
              'head': 5,
              'lemma': '_',
              'rel': 'nsubjpass',
              'tag': '_',
              'word': 'correspondence'},
             3: {'address': 3,
              'cta

In [19]:
list(dg.triples())

[(('use', 'VERB'), 'nsubjpass', ('correspondence', 'NOUN')),
 (('correspondence', 'NOUN'), 'det', ('this', 'DET')),
 (('use', 'VERB'), 'aux', ('can', 'VERB')),
 (('use', 'VERB'), 'auxpass', ('be', 'VERB')),
 (('use', 'VERB'), 'prep', ('for', 'ADP')),
 (('for', 'ADP'), 'pcomp', ('provide', 'VERB')),
 (('provide', 'VERB'), 'dobj', ('property', 'NOUN')),
 (('property', 'NOUN'), 'det', ('a', 'DET')),
 (('property', 'NOUN'), 'amod', ('direct', 'ADJ')),
 (('property', 'NOUN'), 'amod', ('interpretationof', 'NOUN')),
 (('property', 'NOUN'), 'compound', ('markov', 'PROPN')),
 (('provide', 'VERB'), 'punct', (',', 'PUNCT')),
 (('provide', 'VERB'), 'prep', ('in', 'ADP')),
 (('in', 'ADP'), 'advmod', ('both', 'CCONJ')),
 (('in', 'ADP'), 'pobj', ('case', 'NOUN')),
 (('case', 'NOUN'), 'det', ('the', 'DET')),
 (('case', 'NOUN'), 'amod', ('undirected', 'ADJ')),
 (('undirected', 'ADJ'), 'cc', ('and', 'CCONJ')),
 (('undirected', 'ADJ'), 'conj', ('direct', 'VERB')),
 (('provide', 'VERB'), 'punct', (',', 'P

In [94]:
?dg.triples

In [45]:
g = dg.nx_graph()

In [46]:
import networkx

In [55]:
?g.

In [25]:
list(p.noun_chunks)

[One way,
 the natural gradient,
 it,
 a steepestdescent direction,
 the negative gradient,
 respect,
 a metricthat,
 the distributions,
 the default Euclideanmetric,
 parameter space]

In [30]:
tre = p.print_tree()

/usr/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)
/usr/lib/python3.5/runpy.py:184: DeprecationWarning: Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)


In [48]:
[x['lemma'] for x in tre[0]['modifiers']]

['way', 'show', '.']

In [44]:
tre[0]['modifiers'][0]

'be'

In [94]:
dg.nodes[2]

{'address': 2,
 'ctag': 'NOUN',
 'deps': defaultdict(list, {'nummod': [1], 'relcl': [4]}),
 'feats': 'NN',
 'head': 8,
 'lemma': '_',
 'rel': 'nsubj',
 'tag': '_',
 'word': 'way'}

In [92]:
dg.nodes[7]

{'address': 7,
 'ctag': 'NOUN',
 'deps': defaultdict(list, {'amod': [6], 'det': [5]}),
 'feats': 'NN',
 'head': 4,
 'lemma': '_',
 'rel': 'dobj',
 'tag': '_',
 'word': 'gradient'}

In [82]:
a = list(p.noun_chunks)[1]

In [90]:
list(p.noun_chunks)

[One way,
 the natural gradient,
 it,
 a steepestdescent direction,
 the negative gradient,
 respect,
 a metricthat,
 the distributions,
 the default Euclideanmetric,
 parameter space]

In [83]:
a.end

7

In [84]:
a.start

4

In [85]:
a

the natural gradient

In [1]:
import spacy

In [2]:
nlp = spacy.load('en')

In [3]:
text = open('all_sents.txt')

In [4]:
def batches(text):
    lines = []
    for line in text:
        lines.append(line)
        if len(lines) > 3500:
            yield ''.join(lines)
            lines = []
    

In [5]:
f = open('sents_norm_.txt', 'w')
for i, batch in enumerate(batches(text)):
    doc = nlp(batch)
    if not i % 10:
        print('Processed - ', i * 3500)
    for sent in doc.sents:
        f.write(' '.join([word.lemma_ for word in sent if not any([word.is_punct, word.is_space])]) + '\n')
f.close()

Processed -  0
Processed -  35000
Processed -  70000
Processed -  105000
Processed -  140000
Processed -  175000
Processed -  210000
Processed -  245000
Processed -  280000
Processed -  315000
Processed -  350000
Processed -  385000
Processed -  420000
Processed -  455000
Processed -  490000


KeyboardInterrupt: 

In [88]:
f = open('phrases.txt', 'w')

In [89]:
for sent in sents:
    p = nlp(sent)
    f.write('\t'.join([str(x).lower() for x in p.noun_chunks])+'\n')

In [90]:
f.close()

In [91]:
phrases = [x.split('\t') for x in open('phrases.txt')]

In [92]:
len(phrases)

834743

In [93]:
phrases[:50]

[['relative locations', 'major parts', 'the human brain\n'],
 ['the cerebral cortical mantle',
  'about thirty billion neurons',
  'projections',
  'the thalamus',
  'reciprocal projections',
  'the thalamocortical system\n'],
 ['the mantle',
  'three major cortical appendages',
  'the basal ganglia',
  'cerebellum',
  'movement',
  'the hippocampus',
  'memory\n'],
 ['them',
  'the oldest part',
  'the brain',
  'evolutionary terms',
  'several diffusely projecting value systems\n'],
 ['bottom', 'synaptic connections', 'two neurons\n'],
 ['an action potential',
  'the axon',
  'the presynaptic neuron',
  'the release',
  'a neurotransmitter',
  'the synaptic cleft\n'],
 ['the transmitter molecules',
  'the postsynaptic membrane',
  'the probability',
  'the postsynaptic cell',
  'turn',
  'its own action potential\n'],
 ['particular sequences', 'activity', 'the synapse', 'its efficacy\n'],
 ['the number',
  'different shapes',
  'kinds',
  'neurons',
  'this drawing',
  'a greatly sim

In [67]:
import gensim

In [68]:
sents = [x.split() for x in open('sents_norm.txt')]

In [10]:
w2v = gensim.models.Word2Vec(sents, size=400, window=3)

In [11]:
w2v_1 = gensim.models.Word2Vec(sents, size=400, window=5)

In [12]:
w2v_2 = gensim.models.Word2Vec(sents, size=500, window=5)

In [23]:
w2v_3 = gensim.models.Word2Vec(sents, size=600, window=5)

In [70]:
ft = gensim.models.fasttext.FastText(sents, size=200, min_n=3, max_n=6)

In [69]:
?gensim.models.fasttext.FastText

In [22]:
w2v.most_similar('accusative', topn=20)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nominative', 0.876392126083374),
 ('dative', 0.7972114086151123),
 ('absolutive', 0.7938792109489441),
 ('genitive', 0.7907388210296631),
 ('ergative', 0.7671347260475159),
 ('oblique', 0.7181569337844849),
 ('ablative', 0.7101474404335022),
 ('lative', 0.6539338231086731),
 ('instrumental', 0.6520599722862244),
 ('comitative', 0.6512896418571472),
 ('allative', 0.6487379670143127),
 ('benefactive', 0.6452040672302246),
 ('locative', 0.6448217630386353),
 ('obviative', 0.6369820237159729),
 ('postposition', 0.6352605223655701),
 ('vocative', 0.6351326107978821),
 ('unmark', 0.6299031376838684),
 ('unmarked', 0.6291846632957458),
 ('case’', 0.6261202096939087),
 ('-m', 0.6224660277366638)]

In [15]:
w2v_1.most_similar('accusative')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nominative', 0.8737972974777222),
 ('dative', 0.8077637553215027),
 ('absolutive', 0.7807317972183228),
 ('genitive', 0.7784498333930969),
 ('ergative', 0.7567505240440369),
 ('oblique', 0.740955650806427),
 ('ablative', 0.6905251741409302),
 ('unmark', 0.6629302501678467),
 ('locative', 0.6494957208633423),
 ('marking', 0.64488685131073)]

In [21]:
w2v_2.most_similar('accusative', topn=20)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('nominative', 0.8805449604988098),
 ('dative', 0.7963292002677917),
 ('genitive', 0.7926821112632751),
 ('absolutive', 0.7780560851097107),
 ('ergative', 0.7724252343177795),
 ('oblique', 0.7572967410087585),
 ('ablative', 0.6995598077774048),
 ('comitative', 0.6681219339370728),
 ('allative', 0.6622324585914612),
 ('instrumental', 0.6574176549911499),
 ('case’', 0.650806188583374),
 ('unmark', 0.6372339725494385),
 ('antipassive', 0.6340838074684143),
 ('locative', 0.6323443651199341),
 ('marking', 0.6321450471878052),
 ('benefactive', 0.630612850189209),
 ('-m', 0.6254040598869324),
 ('ofnominative', 0.6181294918060303),
 ('vocative', 0.6134778261184692),
 ('postposition', 0.6074748635292053)]

In [66]:
w2v.most_similar('sis', topn=20)

[('-ro', 0.8631316423416138),
 ('ita', 0.8414170145988464),
 ('-tu', 0.8384491205215454),
 ('ι', 0.8383947014808655),
 ('emo', 0.8338407874107361),
 ('p3sg', 0.8324854969978333),
 ('-je', 0.8286639451980591),
 ('no=', 0.8277689814567566),
 ('fal', 0.8269727826118469),
 ('tºi', 0.8266384601593018),
 ('dy', 0.8252228498458862),
 ('-man', 0.8245456218719482),
 ('o-', 0.8240092992782593),
 ('gef', 0.8220754861831665),
 ('tə', 0.8220458626747131),
 ('eti', 0.82012540102005),
 ('ké', 0.8194481134414673),
 ('gv', 0.816381573677063),
 ('há', 0.8160685896873474),
 ('oand', 0.8147521615028381)]

In [31]:
ft.most_similar('accusative', topn=20)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('−accusative', 0.986430823802948),
 ('nominativeaccusative', 0.9578328132629395),
 ('unaccusative', 0.9475518465042114),
 ('accusative’', 0.9464986324310303),
 ('accusatif', 0.9159835577011108),
 ('acusativo', 0.9132360219955444),
 ('dative', 0.9010818004608154),
 ('unaccusative’', 0.8966334462165833),
 ('accusatively', 0.888617217540741),
 ('ergativeabsolutive', 0.8863722681999207),
 ('ergative', 0.8788683414459229),
 ('nominative', 0.8772831559181213),
 ('unergative', 0.8729730844497681),
 ('ofnominative', 0.8710689544677734),
 ('ofdative', 0.8701499104499817),
 ('unaccusatives', 0.8688222765922546),
 ('genitive', 0.865285336971283),
 ('akkusativ', 0.8575778007507324),
 ('accusativity', 0.8533416390419006),
 ('nominative’', 0.8457307815551758)]

In [29]:
ft2 = gensim.models.fasttext.FastText(sents, size=400, window=5, min_count=5)

In [32]:
ft2.most_similar('case', topn=20)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('\x01case', 0.9412144422531128),
 ('•\x01case', 0.9404243230819702),
 ('þcase', 0.9396320581436157),
 ('^case', 0.9392638206481934),
 ('⟨{case', 0.9388969540596008),
 ('〈{case', 0.9380385279655457),
 ('k[ucase', 0.9280897974967957),
 ('case2', 0.9276347756385803),
 ('ucase', 0.9266526699066162),
 ("case'", 0.9232484698295593),
 ('case1', 0.9217522144317627),
 ('casep', 0.9107418656349182),
 ('ofcase', 0.9086319804191589),
 ('caseg', 0.9021592736244202),
 ('caseagr', 0.8774917125701904),
 ('cases’', 0.8653406500816345),
 ('suitcase', 0.838678240776062),
 ('casey', 0.8356001973152161),
 ('subcase', 0.8287577629089355),
 ('casert', 0.8243017792701721)]

In [20]:
from collections import Counter
from nltk.corpus import stopwords
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
stops = stopwords.words('english')

In [8]:
phrases = Counter()
for line in open('phrases.txt'):
    ph = line.strip('\n').split('\t')
    phrases.update(ph)

In [9]:
filtered = set()
for phrase in phrases:
    if phrases[phrase] > 10 and phrase not in stops:
        filtered.add(phrase)

In [51]:
w2v_3.save('w2v_600')

In [4]:
w2v = gensim.models.Word2Vec.load('w2v_600')

In [52]:
ordered = list(filtered)

In [82]:
X = np.zeros((41627, 200))
for i, phrase in enumerate(ordered):
    norm = nlp(phrase)
    words = [w.lemma_ for w in norm if not w.is_stop and not w.is_punct]
    if words:
        try:
            v = ft[' '.join(words)]
        except KeyError:
            continue
#         if v:
#             v = np.mean(v, axis=0)
        X[i] = v

In [18]:
np.mean([w2v['l'], w2v['s'], w2v['x']], axis=0).shape

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(600,)

In [83]:
from sklearn.cluster import MiniBatchKMeans
from collections import defaultdict

In [84]:
km = MiniBatchKMeans(3000,)

In [85]:
km.fit(X)

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=3000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=0)

In [30]:
?MiniBatchKMeans

In [86]:
labels = km.labels_

In [76]:
X

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.47148389,  0.93895769, -0.9723177 , ..., -3.79149342,
         1.52883148, -0.75833899],
       [ 2.56352687, -1.2912581 ,  0.14725791, ...,  0.43182001,
         0.52801687,  0.60488158],
       ...,
       [-1.6909157 ,  0.15794948,  0.32625058, ..., -0.44088709,
         1.10764575, -2.94832015],
       [ 0.26946443,  0.0091242 ,  0.73126578, ..., -0.13193665,
         0.62722892, -0.29373828],
       [-0.01878791,  1.073524  ,  0.53372538, ..., -1.30288756,
         1.27855051, -1.04604042]])

In [75]:
labels

array([1185,  339,   73, ...,  433,  882, 1927], dtype=int32)

In [77]:
lb_dict[1185]

['jared s. klein', 'matthew s. dryer', 's. goldin-meadow']

In [87]:
lb_dict = defaultdict(list)
for i, phrase in enumerate(ordered):
    lb_dict[labels[i]].append(phrase)

In [88]:
f = open('clusters.txt', 'w')
for cl in lb_dict:
    if cl != 449:
        f.write(str(cl) + '\n')
    f.write('\t'.join(lb_dict[cl]))
    f.write('\n\n\n')
f.close()

In [51]:
norm[0].is_stop

True

In [26]:
a = open('opendata/all_dois_20180122T165326.csv')

In [39]:
a.readline()

'10.1016/0167-4781(85)90058-2,f,2,,,,,,,,Normal rate of DNA breakage in xeroderma pigmentosum complementation group E cells treated with 8-methoxypsoralen plus near-ultraviolet radiation,0167-4781,Biochimica et Biophysica Acta (BBA) - Gene Structure and Expression,f,Elsevier BV,1985,journal-article,2018-01-21 07:37:13.422973\n'

In [8]:
import csv

In [ ]:
f = open('opened.csv', 'w')

In [46]:
with open('opened.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields, delimiter=',', quotechar='"')
    writer.writeheader()


    with open('opendata/all_dois_20180122T165326.csv', newline='\n') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')

        for line in reader:
            if line['is_oa'] == 't':
                writer.writerow(line)
            

In [47]:
def check(x):
    keys = ['language', 'pronunciation', 'linguist', 'syntact', 'syntax', 'morpholog', 'semantic',
           'clitic', 'gramma', 'phonolog', 'lexi', 'phonetic', 'translation', 'discourse',
           'verb', 'noun', 'adjective', 'adverb', 'pronoun', 'lingua', 'broca', 'wernick',
           'pronominal', 'speech', 'creol', 'syllab', 'prosod', 'aphasia', 'spelling', 'text',
           'predicat', 'spoken', 'communic', 'metaphor', 'consonant', 'vowel', 'phonem', 'chomsky',
           'word', 'vocabular', 'accusativ', 'dative', 'genetiv', 'phrase', 'preposition', 'dialect',
           'corpus', 'corpora']
    x = x.lower()
    for key in keys:
        if key in x:
            return True
    else:
        return False

In [49]:
i = 0
with open('ling_possible.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=fields, delimiter=',', quotechar='"')
    writer.writeheader()
    with open('opened.csv') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')

        for line in reader:
            if check(line['title']):
                writer.writerow(line)
                i += 1

In [54]:
import requests
import os
import grequests

In [57]:
reqs = set()

In [59]:
i = 0
errors = []
downloaded = set([x.replace('#', '/').split('.pdf')[0] for x in os.listdir('opendata/linguistic')])

with open('ling_possible.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"', fieldnames=fields)
    reqs2sent = []
    for jj, line in enumerate(reader):
        if jj < j:
            continue
        if line['doi'] in downloaded:
#             print('skipping')
            continue
        
        if not 'lingu' in line['title'] and not 'lingu' in line['journal_name']:
            continue
        url = line['best_oa_location_url_for_pdf']
        if url and url not in reqs:
#             if len(reqs2sent) > 100:
#                 rs = (grequests.get(u, timeout=1) for u in reqs2sent)
#                 rs = grequests.map(rs)
#                 reqs2sent = []
#             else:
#                 reqs2sent.append(url)
#                 reqs.add(url)
#                 continue
            
#             for r in rs:
#                 if not r:
#                     continue
            try:
                r = requests.get(url, timeout=5)
                reqs.add(url)
            except Exception as e:
                errors.append((url, e))
#                 print(e)
                continue
            typ = r.headers.get('content-type')
            if not typ:
                continue

            if r.status_code == 200 and 'application/pdf' in typ:
                nf = open('opendata/linguistic//{}.pdf'.format(line['doi'].replace('/', '#')), 'wb')
                nf.write(r.content)
                nf.close()
                downloaded.add(line['doi'])
                i += 1
                if not i % 1000:
                    print('Downloaded - ', i)
            else:
                continue


Downloaded -  1000
Downloaded -  2000
Downloaded -  3000


In [50]:
rs = (grequests.get(u, timeout=1) for u in reqs2sent)

In [51]:
rrs = grequests.map(rs)

In [52]:
rrs

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [503]>,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [404]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 None,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [403]>,
 None,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [503]>,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 None,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [404]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 <Response [403]>,
 <Response [200]>,
 None,
 <Response [404]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Re

In [41]:
rrs

[<Response [200]>,
 None,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 <Response [403]>,
 <Response [404]>,
 <Response [403]>,
 <Response [200]>,
 <Response [403]>,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 None,
 None,
 <Response [404]>,
 <Response [403]>,
 <Response [200]>,
 None,
 <Response [404]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 <Response [200]>,
 <Response [404]>,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [403]>,
 <Response [200]>,
 None,
 None,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [403]>,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [403]>,
 <Response [200]>,
 <Response [200]>,
 <Response [403]>,
 <Response [403]>,
 <Response [20

In [23]:
j = jj

In [12]:
j = 0
with open('ling_possible.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        if row['doi'] in downloaded:
            j = i
        

In [13]:
j

20205

In [76]:
r.headers.get('content-type')

'text/html;charset=UTF-8'

In [73]:
downloaded

{'10.1002/9781118584194',
 '10.1002/aqc.2675',
 '10.1002/ejsp.240',
 '10.1002/humu.10023',
 '10.1002/neu.20096',
 '10.1002/yea.1503',
 '10.1007/11669487_3',
 '10.1007/11761679_34',
 '10.1007/11914952_47',
 '10.1007/978-3-319-02762-3_13',
 '10.1007/978-3-319-19662-6_15',
 '10.1007/978-3-319-21365-1_28',
 '10.1007/978-3-319-26647-3_133',
 '10.1007/978-3-476-03163-1_2',
 '10.1007/978-3-540-30213-1_15',
 '10.1007/978-3-540-30557-6_9',
 '10.1007/978-3-540-45209-6_144',
 '10.1007/978-3-540-72794-1_4',
 '10.1007/978-3-540-74456-6_50',
 '10.1007/978-3-540-78135-6_30',
 '10.1007/978-3-642-11284-3_7',
 '10.1007/978-3-642-20000-7_7',
 '10.1007/978-3-642-24455-1_2',
 '10.1007/978-3-642-40395-8_15',
 '10.1007/978-3-642-41335-3_37',
 '10.1007/978-3-662-49674-9_50',
 '10.1007/bf00170207',
 '10.1007/bf03005076',
 '10.1007/bf03168750',
 '10.1007/bf03168757',
 '10.1007/bfb0021144',
 '10.1007/s00125-006-0212-9',
 '10.1007/s10344-015-0927-3',
 '10.1007/s10681-015-1454-8',
 '10.1007/s11191-004-5157-0',
 '1

In [10]:
fields = ['doi', 'is_oa', 'data_standard', 'best_oa_location_url', 
          'best_oa_location_url_for_landing_page', 'best_oa_location_url_for_pdf',
          'best_oa_location_host_type', 'best_oa_location_version',
          'best_oa_location_license', 'best_oa_location_evidence', 'title',
          'journal_issns', 'journal_name', 'journal_is_oa', 'publisher',
          'year', 'genre', 'updated']

In [42]:
r = csv.DictReader(open('opendata/all_dois_20180122T165326.csv'), quotechar='"', fieldnames=fields)

In [43]:
for line in r:
    print(line)
    break

{'best_oa_location_version': 'best_oa_location_version', 'updated': 'updated', 'data_standard': 'data_standard', 'journal_name': 'journal_name', 'best_oa_location_evidence': 'best_oa_location_evidence', 'journal_issns': 'journal_issns', 'doi': 'doi', 'title': 'title', 'best_oa_location_host_type': 'best_oa_location_host_type', 'publisher': 'publisher', 'journal_is_oa': 'journal_is_oa', 'best_oa_location_url_for_pdf': 'best_oa_location_url_for_pdf', 'year': 'year', 'best_oa_location_license': 'best_oa_location_license', 'is_oa': 'is_oa', 'best_oa_location_url_for_landing_page': 'best_oa_location_url_for_landing_page', 'genre': 'genre', 'best_oa_location_url': 'best_oa_location_url'}


In [44]:
next(r)

{'best_oa_location_evidence': '',
 'best_oa_location_host_type': '',
 'best_oa_location_license': '',
 'best_oa_location_url': '',
 'best_oa_location_url_for_landing_page': '',
 'best_oa_location_url_for_pdf': '',
 'best_oa_location_version': '',
 'data_standard': '1',
 'doi': '10.4271/831455',
 'genre': 'proceedings-article',
 'is_oa': 'f',
 'journal_is_oa': 'f',
 'journal_issns': '',
 'journal_name': 'SAE Technical Paper Series',
 'publisher': 'SAE International',
 'title': 'Efforts to Assure Anthropometric Cockpit Compatibility in Navy Aircraft',
 'updated': '2018-01-21 21:50:08.440155',
 'year': '1983'}

In [157]:
a.readline().split(',')

['doi',
 'is_oa',
 'data_standard',
 'best_oa_location_url',
 'best_oa_location_url_for_landing_page',
 'best_oa_location_url_for_pdf',
 'best_oa_location_host_type',
 'best_oa_location_version',
 'best_oa_location_license',
 'best_oa_location_evidence',
 'title',
 'journal_issns',
 'journal_name',
 'journal_is_oa',
 'publisher',
 'year',
 'genre',
 'updated\n']

In [5]:
len(a.readline().split(','))

18

In [143]:
a.readline()

'10.1017/s027226310606027x,f,2,,,,,,,,REEXAMINING THE ROLE OF RECASTS IN SECOND LANGUAGE ACQUISITION,"0272-2631,1470-1545",Studies in Second Language Acquisition,f,Cambridge University Press (CUP),2006,journal-article,2017-12-24 02:50:14.592608\n'

In [147]:
a.readline()

'10.1007/978-3-319-32489-0_2,f,2,,,,,,,,Rotational and Translational Diffusion in Ionic Liquids,"2190-930X,2190-9318",Dielectric Properties of Ionic Liquids,f,Springer International Publishing,2016,book-chapter,2018-01-18 07:23:44.610323\n'

In [151]:
f = open('opened.csv', 'w')
for line in a:
    l = line.split(',')
    if len(l) < 2:
        continue
    if l[1] == 't':
        f.write(line)
f.close()

In [150]:
line

'Internacional de Literatura i Cultura Medieval i\n'

In [155]:
f = open('batch.txt', 'w')
full = open('opened.csv')

for i in range(1000000):
    f.write(full.readline())
f.close()
full.close()

In [1]:
from collections import Counter
from nltk.tokenize import word_tokenize

In [26]:
journals = Counter()
for line in open('opened.csv'):
    j = line.split(',')
    if len(j) < 18:
        continue
    if j[12]:
        journals.update([j[12]])

In [8]:
len(journals)

718541

In [28]:
f = open('journals.txt', 'w')
for j, c in journals.most_common():
    f.write('\t'.join([j,str(c)])+'\n')
f.close()

In [2]:
texts = []
for line in open('batch.txt'):
    t = line.split(',')
    if len(t) < 18:
        continue
    if t[10]:
        texts.append([x for x in word_tokenize(t[10].lower()) if x.isalpha()])


In [3]:
import gensim

In [4]:
dictionary = gensim.corpora.Dictionary(texts)

In [18]:
print(dictionary)

Dictionary(64995 unique tokens: ['integrando', 'hadoop', 'azide', 'interlocked', 'phaeophyceae']...)


In [17]:
dictionary.filter_extremes(no_below=5, keep_n=None)

In [19]:
tfidf = gensim.models.TfidfModel([dictionary.doc2bow(x) for x in texts], id2word=dictionary)

In [20]:
bow = [dictionary.doc2bow(x) for x in texts]

In [8]:
lda = gensim.models.LdaModel(tfidf[bow], 200, id2word=dictionary)

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


KeyboardInterrupt: 

In [24]:
lda = gensim.models.LdaMulticore(tfidf[bow], 200, id2word=dictionary, iterations=20, chunksize=2000, workers=3, alpha=0.01)

/usr/local/lib/python3.5/dist-packages/gensim/models/ldamodel.py:802: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [25]:
lda.print_topics(200)

[(0,
  '0.049*"fibrosis" + 0.046*"matter" + 0.045*"world" + 0.045*"white" + 0.043*"order" + 0.040*"material" + 0.036*"near" + 0.033*"extract" + 0.032*"cystic" + 0.032*"transporter"'),
 (1,
  '0.061*"events" + 0.055*"behaviour" + 0.048*"architecture" + 0.046*"extracellular" + 0.045*"dendritic" + 0.042*"law" + 0.034*"title" + 0.033*"promote" + 0.031*"mobile" + 0.030*"ischemia"'),
 (2,
  '0.053*"modified" + 0.051*"adolescents" + 0.046*"school" + 0.041*"associations" + 0.040*"children" + 0.031*"video" + 0.030*"initiation" + 0.021*"dengue" + 0.019*"among" + 0.017*"and"'),
 (3,
  '0.065*"advanced" + 0.059*"series" + 0.030*"co" + 0.028*"requirements" + 0.028*"node" + 0.027*"speed" + 0.026*"brasil" + 0.025*"focal" + 0.022*"lymph" + 0.021*"silencing"'),
 (4,
  '0.072*"knowledge" + 0.046*"comment" + 0.044*"evolutionary" + 0.043*"elements" + 0.038*"robust" + 0.031*"basic" + 0.030*"set" + 0.028*"functional" + 0.028*"degeneration" + 0.021*"anomalous"'),
 (5,
  '0.106*"networks" + 0.046*"sensor" + 0